<a href="https://colab.research.google.com/github/ChristianFel7/Python-para-Data-Science/blob/main/Spark_trabalhando_com_regress%C3%A3o.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

download da base de dados:
https://cdn3.gnarususercontent.com.br/2277-spark/01/aula01.zip

In [10]:
https://cdn3.gnarususercontent.com.br/2277-spark/01/aula01.zip!pip install findspark

In [11]:
import findspark
findspark.init()

In [12]:
from pyspark.sql import SparkSession

In [13]:
spark = SparkSession.builder \
    .master('local[*]') \
    .appName("Learning Spark") \
    .getOrCreate()

In [14]:
spark

In [ ]:
from google.colab import drive
drive.mount('/content/drive')